# 🧬 Dimensionality Reduction and Unsupervised Clustering for PAM50 Subtypes

This notebook explores a variety of **dimensionality reduction** and **unsupervised clustering** techniques applied to breast cancer gene expression data. The primary goal is to evaluate whether unsupervised methods can recover the known **PAM50 subtypes**.

---


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
df_meta_all = pd.read_csv("./data/df_meta.tsv",sep="\t",index_col=0)
df_zscore = pd.read_csv("./data/df_merged.tsv",sep="\t",index_col=0)
df_zscore_meta = df_meta_all[df_meta_all["sample_id"].isin(list(df_zscore.columns))]

## 📉 Step 2: Dimensionality Reduction Methods

Several dimensionality reduction techniques are used to project high-dimensional expression data into low-dimensional space.

### 2.1 Principal Component Analysis (PCA)

- A linear projection method that captures the largest variance directions.
- Used for both visualization and feature extraction.
- 2D PCA plots help visually assess subtype separability.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Assume df_merged is the processed expression matrix (genes as rows, samples as columns); transpose it
df_for_clustering = df_zscore.T  # After transposing, each row is a sample

# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_for_clustering)

# Subtype labels (for coloring)
subtypes = df_zscore_meta["pam50_subtype"]

# Visualization
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x=pca_result[:, 0], 
    y=pca_result[:, 1],
    hue=subtypes,
    hue_order=["Basel", "LumA", "LumB", "Normal", "Her2"]
)

plt.title('PCA on Processed Gene Expression')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True)
plt.show()


### 2.2 Autoencoder

- A neural network trained to reconstruct input data through a low-dimensional "bottleneck" layer.
- Encoder output is used as compressed features.
- Captures non-linear structure in gene expression.


In [ ]:
# Try dimension reduction using a simple autoencoder
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

# Assume df_for_clustering is the expression matrix with shape (n_samples, n_genes)
input_dim = df_for_clustering.shape[1]
encoding_dim = 2  # Reduce to 2D for visualization

# Define the autoencoder structure
input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='linear')(encoded)

# Define full autoencoder and encoder models
autoencoder = Model(inputs=input_layer, outputs=decoded)
encoder = Model(inputs=input_layer, outputs=encoded)

# Compile and train the autoencoder
autoencoder.compile(optimizer=Adam(), loss='mse')
autoencoder.fit(df_for_clustering, df_for_clustering, epochs=50, batch_size=64, shuffle=True)

# Get the 2D encoded representation
X_encoded = encoder.predict(df_for_clustering)


## 🌐 PHATE for Nonlinear Dimensionality Reduction

PHATE (Potential of Heat-diffusion for Affinity-based Transition Embedding) is a nonlinear dimensionality reduction technique especially well-suited for high-dimensional biological data such as gene expression.

In [ ]:
import phate

phate_operator = phate.PHATE(n_components=2, knn=5)
X_phate = phate_operator.fit_transform(df_for_clustering)


import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(x=X_phate[:,0], y=X_phate[:,1])
plt.title("PHATE projection of expression")
plt.show()

### 2.3 SimCLR for Gene Expression

- Contrastive self-supervised learning adapted to gene expression data.
- Uses gene masking and Gaussian noise as augmentations.
- A projection head is trained using NT-Xent loss.
- Learned 128-dimensional embeddings are used for clustering.

---

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# ----------------- expression matrix -----------------
#    X : (n_samples, n_genes)  – already z-scored / log-scaled
X = np.array(df_for_clustering)                               # ← your numpy array
X_tensor = torch.tensor(X, dtype=torch.float32)

# ----------------- define two augmentations ------------
def aug_dropout(x, p=0.1):
    mask = (torch.rand_like(x) > p).float()
    return x * mask

def aug_gaussian_noise(x, sigma=0.15):
    return x + sigma * torch.randn_like(x)

class ExpressionPairDataset(TensorDataset):
    """Return two differently-augmented views of the same sample."""
    def __getitem__(self, idx):
        x = self.tensors[0][idx]
        return aug_dropout(x), aug_gaussian_noise(x)

pair_ds   = ExpressionPairDataset(X_tensor)
pair_dl   = DataLoader(pair_ds, batch_size=256, shuffle=True, drop_last=True)
import torch.nn as nn
from lightly.models.modules.heads import SimCLRProjectionHead
from lightly.loss import NTXentLoss

class SimCLRNet(nn.Module):
    def __init__(self, dim_in, dim_feat=2, dim_out=2):
        super().__init__()
        # backbone : maps input → feature (latent)  ⟵ 你最终想要的表示
        self.backbone = nn.Sequential(
            nn.Linear(dim_in, 512), nn.ReLU(),
            nn.Linear(512, dim_feat)
        )
        # projection head : feature → contrastive space
        self.projection = SimCLRProjectionHead(dim_feat, dim_feat, dim_out)

    def forward(self, x):
        h = self.backbone(x)
        z = self.projection(h)
        return z

model = SimCLRNet(dim_in=X.shape[1])
loss_fn = NTXentLoss()                 # temperature-scaled InfoNCE
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

for epoch in range(50):
    total_loss = 0
    for x1, x2 in pair_dl:
        x1, x2 = x1.to(device), x2.to(device)
        z1, z2 = model(x1), model(x2)
        loss   = loss_fn(z1, z2)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch:02d}  loss {total_loss/len(pair_dl):.4f}")
model.eval()
with torch.no_grad():
    latent = model.backbone(X_tensor.to(device)).cpu().numpy()   # (n_samples, 128)

In [ ]:
import numpy as np, torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
from lightly.models.modules.heads import SimCLRProjectionHead
from lightly.loss import NTXentLoss
from sklearn.preprocessing import StandardScaler
from collections import Counter

# ------------ 1. Data Preparation --------------------------------------------------
X = np.asarray(df_for_clustering, dtype=np.float32)        # (n_samples, n_genes)
X = StandardScaler().fit_transform(X)                      # Apply z-score normalization

# (Optional) If PAM50 labels are available: used for balanced sampling
y = df_zscore_meta["pam50_subtype"].values                # String labels
label2idx = {lab: i for i, lab in enumerate(np.unique(y))}
y_int = np.array([label2idx[v] for v in y])

tensor_X = torch.tensor(X, dtype=torch.float32)

# ------------ 2. Balanced Sampler --------------------------------------------------
counts = Counter(y_int)
class_weights = torch.tensor([1.0 / counts[i] for i in counts], dtype=torch.float32)
sample_weights = class_weights[y_int]
sampler = WeightedRandomSampler(weights=sample_weights,
                                num_samples=len(sample_weights),
                                replacement=True)

# ------------ 3. Expression-specific Data Augmentation -----------------------------
def aug_mask(x, p=0.2):          # Randomly mask gene features
    mask = (torch.rand_like(x) > p).float()
    return x * mask

def aug_gaussian(x, sigma=0.1):  # Add Gaussian noise
    return x + sigma * torch.randn_like(x)

class ExprPair(TensorDataset):
    def __getitem__(self, idx):
        x = self.tensors[0][idx]
        return aug_mask(x), aug_gaussian(x)

pair_ds = ExprPair(tensor_X)
pair_dl = DataLoader(pair_ds, batch_size=256, sampler=sampler, drop_last=True, num_workers=0)

# ------------ 4. SimCLR Network ----------------------------------------------------
class SimCLRNet(nn.Module):
    def __init__(self, dim_in, dim_feat=128, dim_out=128):
        super().__init__()
        self.backbone = nn.Sequential(
            nn.Linear(dim_in, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, dim_feat)
        )
        self.projection = SimCLRProjectionHead(dim_feat, dim_feat, dim_out)

    def forward(self, x):
        h = self.backbone(x)
        z = self.projection(h)
        return z

# Initialize model
model = SimCLRNet(dim_in=X.shape[1]).to('cuda' if torch.cuda.is_available() else 'cpu')
loss_fn = NTXentLoss(temperature=0.2)      # Lower temperature for better separation
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=50)

# ------------ 5. Training ----------------------------------------------------------
device = next(model.parameters()).device
for epoch in range(1, 51):
    model.train()
    epoch_loss = 0
    for x1, x2 in pair_dl:
        x1, x2 = x1.to(device), x2.to(device)
        z1, z2 = model(x1), model(x2)
        loss = loss_fn(z1, z2)

        opt.zero_grad()
        loss.backward()
        opt.step()
        epoch_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch:02d}  loss {epoch_loss/len(pair_dl):.4f}")

# ------------ 6. Extract Latent Representations ------------------------------------
model.eval()
with torch.no_grad():
    latent = model.backbone(tensor_X.to(device)).cpu().numpy()   # (n_samples, 128)

# `latent` can now be used for downstream clustering (e.g. KMeans, GMM, Spectral)


## 🔗 Step 3: Unsupervised Clustering Methods

The latent features are clustered using various methods:

### 3.1 K-Means

- Simple and efficient.
- Requires predefining the number of clusters.
- Assumes spherical clusters.

### 3.2 Gaussian Mixture Models (GMM)

- A probabilistic clustering model.
- Learns soft assignments based on multivariate Gaussians.

### 3.3 Spectral Clustering

- Constructs an affinity graph from samples.
- Clustering is performed in the eigenvector space.
- Captures complex, non-linear structures.

### 3.4 HDBSCAN

- Density-based clustering.
- Automatically infers number of clusters.
- Handles outliers and noise effectively.

---

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hdbscan
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_biclusters
from sklearn.cluster import SpectralBiclustering, SpectralCoclustering
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist
from sklearn.cluster import SpectralClustering

# -------------------------------------
# Dimensionality Reduction using PCA
# -------------------------------------
# Transpose the z-score normalized matrix: samples as rows
pca = PCA(n_components=50)
X_pca = pca.fit_transform(df_zscore.T)

# -------------------------------------
# KMeans Clustering (Optional)
# -------------------------------------
# kmeans = KMeans(n_clusters=5, random_state=42)
# clusters = kmeans.fit_predict(df_merged.T)

# -------------------------------------
# Gaussian Mixture Model (Optional)
# -------------------------------------
# n_clusters = 5  # Try values between 3 and 8
# gmm = GaussianMixture(n_components=n_clusters, covariance_type='full', random_state=42)
# clusters = gmm.fit_predict(latent)

# -------------------------------------
# Spectral Biclustering (Optional)
# -------------------------------------
# expr_var = df_merged.var(axis=1)
# top_genes = expr_var.sort_values(ascending=False).head(2000).index
# df_hvg = df_merged.loc[top_genes]
# model = SpectralBiclustering(n_clusters=5, method='log', random_state=0)
# model.fit(df_hvg.T)
# clusters = model.row_labels_

# -------------------------------------
# HDBSCAN Clustering (Optional)
# -------------------------------------
# clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=10)
# clusters = clusterer.fit_predict(df_zscore.T)

# -------------------------------------
# Hierarchical Clustering (Optional)
# -------------------------------------
# distance_matrix = pdist(X_phate, metric='euclidean')
# Z = linkage(distance_matrix, method='ward')
# clusters = fcluster(Z, t=5, criterion='maxclust')

# -------------------------------------
# Spectral Clustering (used here)
# -------------------------------------
n_clusters = 5  # Adjust as needed for desired number of subtypes
sc = SpectralClustering(
    n_clusters=n_clusters,
    affinity='nearest_neighbors',     # Alternative: 'rbf'
    n_neighbors=10,                   # 10–30 is common for gene expression
    assign_labels='kmeans',           # Also try 'discretize'
    random_state=42
)

clusters = sc.fit_predict(X_phate)  # Assuming X_phate is defined earlier


## 📈 Step 4: Visualization

- 2D scatter plots (e.g., PCA) showing predicted clusters.
- Stacked bar plots showing subtype composition within each cluster.
- Helps interpret how well clustering corresponds to known biology.

---


In [ ]:
# === Visualize clustering results on PCA projection ===
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clusters, palette='Set1')
plt.title("Unsupervised Clustering (Spectral) on Gene Expression Data")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()

# === Compare predicted clusters with true PAM50 subtypes ===

# Assign cluster labels to metadata
df_zscore_meta["Cluster"] = clusters

# Count how many samples of each PAM50 subtype appear in each cluster
df_counts = df_zscore_meta.groupby(["Cluster", "pam50_subtype"]).size().reset_index(name="count")

# Compute total number of samples in each cluster
df_totals = df_counts.groupby("Cluster")["count"].transform("sum")

# Add a proportion column for stacked bar plot
df_counts["proportion"] = df_counts["count"] / df_totals

# Pivot table for plotting (clusters as rows, subtypes as columns)
pivot_df = df_counts.pivot(index="Cluster", columns="pam50_subtype", values="proportion")

# Plot stacked bar chart showing subtype composition per cluster
pivot_df.plot(kind="bar", stacked=True, figsize=(8, 5), colormap="Set2")

plt.ylabel("Proportion")
plt.title("Proportion of PAM50 Subtypes within Each Cluster")
plt.legend(title="PAM50 Subtype", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


## 📊 Step 5: Clustering Evaluation

Several metrics are used to compare predicted clusters to true PAM50 subtypes:

- **ARI** (Adjusted Rand Index)
- **NMI** (Normalized Mutual Information)
- **AMI** (Adjusted Mutual Information)
- **Homogeneity / Completeness / V-measure**
- **Silhouette Score** (higher is better)
- **Davies-Bouldin Index** (lower is better)
- **Hungarian Matching Accuracy** for optimal alignment of predicted and true labels

---


In [ ]:
# === Evaluate clustering performance ===

from sklearn.metrics import (
    adjusted_rand_score, 
    normalized_mutual_info_score, 
    adjusted_mutual_info_score, 
    homogeneity_completeness_v_measure,
    silhouette_score, 
    davies_bouldin_score,
    confusion_matrix, 
    accuracy_score
)
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import linear_sum_assignment
import numpy as np

# === 1. Convert ground truth PAM50 labels to integers ===
y_true = LabelEncoder().fit_transform(df_zscore_meta["pam50_subtype"])  # true PAM50 subtype labels
y_pred = clusters  # predicted cluster labels from KMeans / GMM / Spectral, etc.

# === 2. External clustering evaluation metrics ===
ari = adjusted_rand_score(y_true, y_pred)  # Adjusted Rand Index
nmi = normalized_mutual_info_score(y_true, y_pred)  # Normalized Mutual Information
ami = adjusted_mutual_info_score(y_true, y_pred)    # Adjusted Mutual Information
h, c, v = homogeneity_completeness_v_measure(y_true, y_pred)  # Homogeneity, Completeness, V-measure

print(f"ARI = {ari:.3f}")
print(f"NMI = {nmi:.3f}")
print(f"AMI = {ami:.3f}")
print(f"Homogeneity = {h:.3f}, Completeness = {c:.3f}, V-measure = {v:.3f}")

# === 3. Internal clustering evaluation metrics ===
sil = silhouette_score(X_encoded, y_pred)  # Measures intra-cluster tightness and inter-cluster separation
db = davies_bouldin_score(X_encoded, y_pred)  # Lower is better

print(f"Silhouette Score = {sil:.3f} (higher is better)")
print(f"Davies-Bouldin Index = {db:.3f} (lower is better)")

# === 4. Cluster Label Alignment Using Hungarian Algorithm ===
# This step is optional but useful to evaluate clustering accuracy by aligning predicted labels with true labels

# Confusion matrix between true and predicted clusters
conf_mat = confusion_matrix(y_true, y_pred)

# Optimal assignment to maximize overlap using Hungarian algorithm
row_ind, col_ind = linear_sum_assignment(-conf_mat)

# Build mapping: predicted → true
mapping = {pred_label: true_label for pred_label, true_label in zip(col_ind, row_ind)}

# Apply mapping to predictions
y_pred_aligned = np.array([mapping[label] for label in y_pred])

# Compute aligned accuracy
acc = accuracy_score(y_true, y_pred_aligned)
print(f"Cluster Accuracy (after Hungarian alignment): {acc:.3f}")


## ✅ Summary

This notebook demonstrates how **unsupervised learning** and **dimensionality reduction** can be combined to uncover biologically meaningful clusters from breast cancer transcriptomic data. The recovered clusters are evaluated against **PAM50 subtypes**, providing insights into the effectiveness of each method.
